In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
haskey(ENV, "LD_LIBRARY_PATH") && @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "CodecZlib",
    "FASTX"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
# 20240918.nr.faa.gz

In [ ]:
# blast_dbs_dir = joinpath([ENV["SCRATCH"], "workspace", "blastdb"])
blast_dbs_dir = joinpath([homedir(), "workspace", "blastdb"])

In [ ]:
# map to blast NR
blast_db = "20240918.nr"
blast_db_path = joinpath(blast_dbs_dir, blast_db)
blast_db_fasta = blast_db_path * ".faa.gz"
@assert isfile(blast_db_fasta)

In [ ]:
# map to blast NT
# blast_db = "20240418.nt"
# blast_db_path = joinpath(blast_dbs_dir, blast_db)
# blast_db_fasta = blast_db_path * ".fna.gz"
# @assert isfile(blast_db_fasta)

# path_to_db = joinpath(homedir(), "workspace", "blastdb", blast_db)


In [ ]:
function normalize_fasta_for_setencepience(; in_fasta, output = replace(in_fasta, r"\.gz$" => ".normalized.txt.gz"), buffer_size = 1024 * 1024)
    n_records = 0
    n_residues = 0
    @show output
    @assert occursin(r"\.gz$", output)

    open(output, "w") do io
        gzio = CodecZlib.GzipCompressorStream(io)
        buffer = IOBuffer()
        reader = Mycelia.open_fastx(in_fasta)
        # Mycelia.open_fastx(in_fasta) do reader
        for record in reader
            seq = FASTX.sequence(record)
            normalized_seq = uppercase(seq)
            write(buffer, normalized_seq, "\n")
            n_records += 1
            n_residues += length(normalized_seq)

            if bytesavailable(buffer) >= buffer_size
                write(gzio, take!(buffer))
            end
        end
        close(reader)

        if bytesavailable(buffer) > 0
            write(gzio, take!(buffer))
        end

        close(gzio)
    end

    @show n_records
    @show n_residues
    return output
end

@time normalize_fasta_for_setencepience(in_fasta = blast_db_fasta)

In [ ]:
# function normalize_fasta_for_setencepience(;in_fasta, output=replace(in_fasta, r"\.gz$" => ".normalized.txt.gz"))
#     n_records = 0
#     n_residues = 0
#     @show output
#     @assert occursin(r"\.gz$", output)
#     open(output, "w") do io
#         gzio = CodecZlib.GzipCompressorStream(io)
#         for record in Mycelia.open_fastx(in_fasta)
#             seq = FASTX.sequence(record)
#             normalized_seq = uppercase(seq)
#             println(gzio, normalized_seq)
#             n_records += 1
#             n_residues += length(normalized_seq)
#         end
#         close(gzio)
#     end
#     @show n_records
#     @show n_residues
#     return output
# end

# @time normalize_fasta_for_setencepience(in_fasta = blast_db_fasta)

In [ ]:
# dbs = [
#     "nr",
#     "nt"
# ]

In [ ]:
# for db in dbs
#     @show db
#     # use NCBI so that we only re-download the database fragments that have changed
#     path_to_db = Mycelia.download_blast_db(db=db, source="ncbi")
#     # write out to fna - note this doesn't check to see if the fasta is out of date - but it should
#     Mycelia.export_blast_db(path_to_db = path_to_db)
# end